# Tool for manipulating tensors using the hierarchical fiber abstraction

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

# Uncomment the first line and comment out the second to show movie animations

#%run ../prelude.py
%run ../prelude.py --no-show-animations


## Creating a tensor

Following is an example of reading in a tensor from a file in YAML format.


In [ ]:
        
# Display an example tensor

filename = datafileName("draw-a.yaml")

print("YAML represenation of a tensor\n")
f = open(filename)
for line in f:
    print(line.rstrip('\n'))
f.close()

## Create and display a tensor from a YAML file

In [ ]:
a = Tensor.fromYAMLfile(filename)

print("Fiber-tree picture of a tensor")
displayTensor(a)

## Print output for fibers in the tensor

In [ ]:
# Get the root fiber out of the tensor
a_m = a.getRoot()

print("Formatted printout of fiber\n")
print(f"{a_m}\n\n")

print("Formatted printout of fiber (with newlines)\n")
print(f"{a_m:n}\n\n")

print("Formatted printout of fiber (with newlines and no elipsis)\n")
print(f"{a_m:n*}\n\n")

print("Formatted printout of fiber (with explicit coordinate and payload format)\n")
print(f"{a_m:(02d,03d)n*}\n\n")


## Create a tensor from an uncompressed array

One can also create a tensor from an set of nested lists

In [ ]:
b_data = [[0, 0, 0, 60, 70, 0, 0, 0],
          [0, 0, 0, 0, 70, 80, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 90, 100]]

b = Tensor.fromUncompressed(["X", "Y"], b_data)

displayTensor(b)

## Traverse a tensor

The fibers in a tensor (starting with the root fiber) can be interated over using a for loop. Each iteration returns the coordinate and payload for each element in the fiber. If the payload is itself a fiber then that fiber can be iterated over.

In [ ]:
# Traverse a tensor

a = Tensor.fromYAMLfile(datafileName("matrix-a.yaml"))

displayTensor(a)

In [ ]:

canvas = createCanvas(a)

a_m = a.getRoot()

for m, (a_k) in a_m:
    print(f"({m}, {a_k})")
    for k, (a_val) in a_k:
        print(f"Processing: ({k}, {a_val})")
        addFrame(canvas, (m,k))

displayCanvas(canvas)

# Element-wise update (empty) tensor, i.e., copy

In [ ]:
# Element-wise update a tensor

a = Tensor.fromYAMLfile(datafileName("elementwise-a.yaml"))
z = Tensor(rank_ids=["M"])

a_m = a.getRoot()
z_m = z.getRoot()

print("Z < A Fiber")

canvas = createCanvas(a, z)

for m, (z_ref, a_val) in z_m << a_m:
    print(f"Processing: ({m}, ({z_ref}, {a_val})")
    
    z_ref += a_val
    addFrame(canvas, [m], [m])

displayCanvas(canvas)

# Intersection

In [ ]:
# Fiber instersection

a = Tensor.fromYAMLfile(datafileName("elementwise-a.yaml"))
b = Tensor.fromYAMLfile(datafileName("elementwise-b.yaml"))

a_m = a.getRoot()
b_m = b.getRoot()

print("Fiber a_m & b_m")

z_m = a_m & b_m

canvas = createCanvas(a, b, z_m)
displayCanvas(canvas, width=None)

## Element-wise multiply

In [ ]:
# Element-wise multiply

a = Tensor.fromYAMLfile(datafileName("elementwise-a.yaml"))
b = Tensor.fromYAMLfile(datafileName("elementwise-b.yaml"))
z = Tensor(rank_ids=["M"])

a_m = a.getRoot()
b_m = b.getRoot()
z_m = z.getRoot()

print("Z < A Fiber")

canvas = createCanvas(a, b, z)

for m, (z_ref, (a_val, b_val)) in z_m << (a_m & b_m):
    print(f"Processing: ({m}, ({z_ref}, ({a_val}, {b_val})))")

    z_ref += a_val * b_val
    addFrame(canvas, (m,), (m,), (m,))

displayCanvas(canvas, width="75%")

## Dot-product

Here is a dot product of two tensors

In [ ]:
# Dot product
#
# To perform a dot-product we need a "row" for an output.
# So we represent the vectors as 2-D tensors
#


a = Tensor.fromYAMLfile(datafileName("dot-product-a.yaml"))
b = Tensor.fromYAMLfile(datafileName("dot-product-b.yaml"))
z = Tensor(rank_ids=["M"])

a_m = a.getRoot()
b_m = b.getRoot()
z_m = z.getRoot()

canvas = createCanvas(a, b, z)

for m, (z_ref, (a_k, b_k)) in z_m << (a_m & b_m):
    for k, (a_val, b_val) in a_k & b_k:
        print(f"Processing: [{k} -> ( {z_ref}, ({a_val}, {b_val})]")

        z_ref += a_val * b_val
        addFrame(canvas, (m,k), (m, k), (m,))


displayCanvas(canvas, width="50%")

# Union

In [ ]:
# Fiber union

a = Tensor.fromYAMLfile(datafileName("elementwise-a.yaml"))
b = Tensor.fromYAMLfile(datafileName("elementwise-b.yaml"))

displayTensor(a)
displayTensor(b)

In [ ]:
a_m = a.getRoot()
b_m = b.getRoot()

print("Fiber a_m | b_m")

z_m = a_m | b_m

displayTensor(z_m)

## Element-wise addition

Following...



In [ ]:
#
# Do a sum of sums of the rows of two matrices
#

a = Tensor.fromYAMLfile(datafileName("dot-product-a.yaml"))
b = Tensor.fromYAMLfile(datafileName("dot-product-b.yaml"))

z = Tensor(rank_ids=["M"])

a_m = a.getRoot()
b_m = b.getRoot()
z_m = z.getRoot()

canvas = createCanvas(a, b, z)

for m, (z_ref, (mask_k, a_k, b_k)) in z_m << (a_m | b_m):
    for k, (ab_mask, a_val, b_val) in a_k | b_k:
        print(f"Processing: [{k} -> ( {z_ref}, ({ab_mask}, {a_val}, {b_val})]")

        z_ref += a_val + b_val
        addFrame(canvas, (m, k), (m, k), (m,))


displayCanvas(canvas, width="250")

## Reduce vector to a rank zero tensor

In [ ]:
a = Tensor.fromYAMLfile(datafileName("elementwise-a.yaml"))
z = Tensor(rank_ids=[])

a_m = a.getRoot()
z_ref = z.getRoot()

canvas = createCanvas(a, z)

for m_coord, (a_val) in a_m:
    z_ref += a_val
    addFrame(canvas, (m_coord,), [])

displayCanvas(canvas, width=None)

## Testing area

For running alternative algorithms